In [ ]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Import And Preprocess Data

In [ ]:
import zipfile
import os
train_zip_path = '/content/drive/My Drive/action_recog_dataset.zip'
# Unzip train.zip
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/new_action_recog_dataset')

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [ ]:
actions = np.array(['ILoveYou', 'Peace', 'Thanks'])
label_map = {label:num for num, label in enumerate(actions)}
label_map
{'ILoveYou': 0, 'Peace': 1, 'Thanks': 2}


{'ILoveYou': 0, 'Peace': 1, 'Thanks': 2}

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content/action_recog_dataset')
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
DATA_PATH

'/content/action_recog_dataset'

In [ ]:
 sorted(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in sorted(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int)):
        window = []
        for frame_num in range(1,1+sequence_length):

            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)


            sequences.append(window)
            labels.append(label_map[action])



In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True)

In [ ]:
import numpy as np

# Define save path
save_path = "/content/drive/My Drive/saved_dataset/"

# Save training and testing data
np.save(save_path + "X_train.npy", X_train)
np.save(save_path + "X_test.npy", X_test)
np.save(save_path + "y_train.npy", y_train)
np.save(save_path + "y_test.npy", y_test)


In [ ]:
# Load training and testing data
save_path = "/content/drive/My Drive/saved_dataset/"

X_train = np.load(save_path + "X_train.npy")
X_test = np.load(save_path + "X_test.npy")
y_train = np.load(save_path + "y_train.npy")
y_test = np.load(save_path + "y_test.npy")


KeyboardInterrupt: 

# Building LSTM Neural Network And Training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, Callback

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
class StopAtAccuracy(Callback):
    def __init__(self, target_accuracy, min_epochs=100):
        super(StopAtAccuracy, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        acc = logs.get('categorical_accuracy')  # Ensure correct metric
        if acc is not None and acc >= self.target_accuracy:
            print(f"\nStopping training: Categorical Accuracy reached {acc:.4f} at epoch {epoch+1}")
            self.model.stop_training = True


In [ ]:
stop_callback = StopAtAccuracy(target_accuracy=0.99)

In [ ]:
model.load_weights('drive/MyDrive/action_recog_weights.weights.h5')

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[stop_callback])


Epoch 1/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - categorical_accuracy: 0.9618 - loss: 0.1204
Epoch 2/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - categorical_accuracy: 0.9696 - loss: 0.0950
Epoch 3/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - categorical_accuracy: 0.9290 - loss: 0.1747
Epoch 4/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - categorical_accuracy: 0.9359 - loss: 0.1540
Epoch 5/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - categorical_accuracy: 0.9003 - loss: 0.2484
Epoch 6/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - categorical_accuracy: 0.8908 - loss: 0.2788
Epoch 7/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - categorical_accuracy: 0.9576 - loss: 0.1213
Epoch 8/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - categorical_accuracy: 0.9380 - loss: 0.1787
Epoch 9/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - categorical_accuracy: 0.9507 - loss: 0.1513
Epoch 10/2000
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - categorical_accuracy: 0.8731 - loss: 0.3472
Epoch 11/

# Predictions On Test Dataset

In [ ]:
res = model.predict(X_test)
actions[np.argmax(res[23])]


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 309ms/step


'ILoveYou'

In [ ]:
actions[np.argmax(y_test[23])]

'ILoveYou'

# Analysis Using Confusion Matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


array([[[ 93,   0],
        [  0,  42]],

       [[101,   0],
        [  0,  34]],

       [[ 76,   0],
        [  0,  59]]])

In [ ]:
import numpy as np
from collections import Counter

# Convert one-hot encoding to class indices
y_test_labels = np.argmax(y_train, axis=1)

# Now use Counter
label_counts = Counter(y_test_labels)
print(label_counts)


Counter({2: 859, 0: 858, 1: 848})


#

In [ ]:
model.save('drive/MyDrive/Action_Recoginition_Model.h5',include_optimizer=True)



In [ ]:
model.save_weights('drive/MyDrive/action_recoginition_Modelweights(!!!!).weights.h5')